## 7. ORQUESTRAÇÃO E CARGA FINAL (Deployment/Implementação)

### 7.1. Justificativa Técnica e Função do main.py

O arquivo main.py é o ponto de entrada (Entry Point) do projeto. Ele move o pipeline de um conjunto de scripts independentes para um Sistema Reprodutível e Gerenciável, alinhado com a fase de Deployment do CRISP-DM.

| Desafio no Projeto | Solução Profissional Implementada | Justificativa |
| :--- | :--- | :---|
| **Execução Manual e Erro Humano** | Orquestração centralizada em main() com parâmetros de linha de comando. | Reprodutibilidade: Garante que o pipeline completo (ETL + Modelagem) seja executado com um único comando, reduzindo erros de execução manual. |
| **Processamento em Etapas** | Chamada sequencial e encadeada dos módulos (etl_antigos, etl_novos, modelagem_dim). | Modularidade: Isola responsabilidades. Se a Modelagem falhar, o ETL não é executado novamente (a menos que forçado). |
| **Carga no BI (Power BI)** | Geração do Star Schema (tabelas Fato e Dimensões) em formato CSV com encoding UTF-8-SIG e separador ponto e vírgula (sép=;). | Compatibilidade: O formato e encoding são o padrão ideal para importação direta no Power BI, garantindo que caracteres especiais e valores numéricos sejam lidos corretamente. |
| **Controle de Versão** | Uso do argparse para gerenciar a flag --force-reprocess (e --analyze-only). | Eficiência: Permite que o usuário opte por reprocessar (ou não) o ETL, economizando tempo se a Modelagem for reexecutada. |
| **Garantia de Fluxo** | Orquestração sequencial e condicional de todos os módulos (ETL Antigo, ETL Novo, Modelagem e Dimensões). | Integridade: Garante que a Modelagem só seja executada após a Limpeza e que o Star Schema só seja criado após o Enriquecimento. |
| **Ambiente Controlado** | Criação inicial das pastas (data/raw, data/outputs, etc.) e tratamento de exceções com try/except. | Robustez: O script garante as pré-condições de ambiente e fornece mensagens de erro claras, facilitando a manutenção. |


### 7.2. Etapas da Orquestração e Carga Final

O arquivo main.py implementa a fase de Deployment (Implementação) do CRISP-DM, garantindo que o ativo de dados seja entregue ao ambiente analítico (Power BI) de forma correta.

| Etapa | Módulo/Função Chamada | Justificativa Técnica |
| :--- | :--- | :--- |
| **Inicialização** | main() / argparse | Define o caminho dos dados (data/), cria as pastas de output e interpreta os comandos do usuário (flags). |
| **ETL - Dados Antigos** | ETLComprasAntigos().processar_todos_antigos() | Carrega e limpa os dados mais complexos (2020-2022), aplicando heurísticas de correção de colunas trocadas. |
| **ETL - Dados Novos** | ETLComprasPublicas().consolidar_todos_anos() | Carrega e limpa os dados recentes (2023-2025), aplicando o leitor flexível. |
| **Consolidação Geral** | processar_e_consolidar_tudo() (internamente) | Une os DataFrames limpos (Antigos + Novos) em uma única Tabela Fato Bruta Consolidada. |
| **Modelagem e Enriquecimento** | modelagem_dim.py (várias funções) | Aplica o Feature Engineering (Z-Score, Risco, Concentração) ao consolidado, gerando o DataFrame Enriquecido. |
| **Carga Final (Deployment)** | dimensoes.criar_e_integrar_dimensoes() | Cria as tabelas Dimensão e a Tabela Fato Final a partir do DataFrame enriquecido e as salva em formato CSV no diretório data/outputs/star_schema/. |

### 7.3. Alinhamento com CRISP-DM e Benefícios Finais

O main.py sintetiza e encerra o ciclo de processamento de dados, entregando o produto final de Data Engineering.

| Fase CRISP-DM | Impacto do main.py | Benefício Entregue ao Projeto | 
| :--- | :--- | :--- |
| **Data Preparation (Final)** | Garante que o schema e os tipos de dados do Star Schema são perfeitamente consistentes para a carga. | Integridade Analítica: Base de dados livre de erros de tipo e compatível com as ferramentas BI. |
| **Modeling (Final)** | Coordena a criação dos indicadores de gestão (Z-Score, Risco, etc.) e sua integração às tabelas. | Ativo Estratégico: Entrega as métricas de negócio pré-calculadas e prontas para visualização. |
| **Deployment** | Orquestra a Carga Final dos arquivos CSV para o Data Warehouse / Ambiente Analítico. | Reprodutibilidade Total: Permite a recriação completa do Star Schema a qualquer momento, essencial para o projeto e para futuras atualizações de dados. | 
| **Manutenção** | O uso de flags (--force-reprocess, --analyze-only) e código modularizado. | Sustentabilidade: Permite que o projeto seja atualizado com novos dados anuais ou refatorado sem quebrar a rotina de processamento. |

#### Benefícios Adicionais da Carga Final

 - Agilidade na Análise: O Star Schema otimiza o tempo de consulta no Power BI de minutos para segundos.

 - Controle de Versão: A separação dos outputs por tabela facilita o controle e a atualização pontual, se apenas uma dimensão mudar.

 - Facilidade de Uso: O analista recebe um conjunto de arquivos CSV limpos (Fato_Compras.csv, Dim_Produto.csv, etc.) que se relacionam de forma intuitiva, sem precisar entender a complexidade do ETL.

### VERSÃO COMENTADA DO MAIN.PY - Para documentação e aprendizado

In [ ]:
# main_explicado.ipynb
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML
import re
import numpy as np

# Configuração de estilo
sns.set_style('whitegrid')
sns.set_palette(["#2E8B57", "#32CD32", "#228B22", "#006400", "#8FBC8F"])

def main_explicado():
    """
     VERSÃO COMENTADA DO MAIN.PY - Para documentação e aprendizado
    """
    
    display(Markdown("#  PIPELINE ETL - VERSÃO EXPLICADA"))
    display(Markdown("##  Guia Interativo do Processamento"))
    
    # =================================================================
    #   ETAPA 1: CONFIGURAÇÃO INICIAL
    # =================================================================
    display(Markdown("## CONFIGURAÇÃO INICIAL"))
    
    display(Markdown("""
    ### Objetivo desta etapa:
    - Criar a estrutura de pastas do projeto
    - Garantir que todos os diretórios necessários existem
    - Configurar caminhos absolutos para evitar erros
    """))
    
    pasta_base = os.path.dirname(os.path.abspath(__file__))
    pasta_dados = os.path.join(pasta_base, "data")
    pasta_raw = os.path.join(pasta_dados, "raw")
    pasta_outputs = os.path.join(pasta_dados, "outputs")

    # Mostrar estrutura de pastas
    display(Markdown("### Estrutura de Pastas Criada:"))
    
    estrutura = f"""
    ```
    {pasta_base}/
    ├── 📄 main.py
    ├── 📄 main_explicado.ipynb
    └── 📂 data/
        ├── 📂 raw/           ←  Arquivos CSV brutos aqui
        └── 📂 outputs/       →  Arquivos processados aqui
    ```
    """
    display(Markdown(estrutura))

    # Criar pastas (simulação)
    for pasta in [pasta_dados, pasta_raw, pasta_outputs]:
        os.makedirs(pasta, exist_ok=True)
        print(f" Pasta criada/verificada: {pasta}")

    # =================================================================
    #   ETAPA 2: DETECÇÃO DE ARQUIVOS
    # =================================================================
    display(Markdown("## DETECÇÃO DE ARQUIVOS"))
    
    display(Markdown("""
    ### Objetivo desta etapa:
    - Verificar se existem arquivos CSV na pasta raw/
    - Identificar automaticamente os anos dos arquivos
    - Separar entre anos antigos (2020-2022) e novos (2023+)
    """))
    
    # Simular arquivos (na realidade, viria de os.listdir)
    arquivos_exemplo = [
        "compras_2020.csv", "compras_2021.csv", "compras_2022.csv",
        "compras_2023.csv", "compras_2024.csv"
    ]
    
    display(Markdown("### Arquivos Detectados:"))
    
    # Criar visualização dos arquivos
    fig, ax = plt.subplots(figsize=(12, 4))
    
    anos = []
    tipos = []
    
    for arquivo in arquivos_exemplo:
        match = re.search(r'20\d{2}', arquivo)
        if match:
            ano = int(match.group())
            anos.append(ano)
            if ano >= 2023:
                tipos.append('NOVO (2023+)')
            else:
                tipos.append('ANTIGO (2020-2022)')

    # Gráfico de distribuição
    unique, counts = np.unique(tipos, return_counts=True)
    ax.bar(unique, counts, color=['#2E8B57', '#32CD32'])
    ax.set_ylabel('Quantidade de Arquivos')
    ax.set_title('Distribuição dos Arquivos por Período')
    plt.xticks(rotation=15)
    plt.tight_layout()
    plt.show()
    
    # Tabela resumo
    resumo_arquivos = pd.DataFrame({
        'Arquivo': arquivos_exemplo,
        'Ano': anos,
        'Tipo': tipos
    })
    display(resumo_arquivos)
    
    # =================================================================
    # ETAPA 3: ESTRATÉGIA DE PROCESSAMENTO
    # =================================================================
    display(Markdown("## ESTRATÉGIA DE PROCESSAMENTO INTELIGENTE"))

    display(Markdown("""
    ### POR QUE processar anos separadamente?
    
    | Período | ETL | Motivo | Vantagem |
    |---------|-----|--------|----------|
    | 2020-2022 | `ETLComprasAntigos` | Formato de dados diferente, estrutura variável | Processamento em **lote** mais eficiente |
    | 2023+ | `ETLComprasPublicas` | Formato padronizado, estrutura consistente | Processamento **individual** com validações específicas |

    **Benefício:** Cada ETL é otimizado para as particularidades do seu período!
    """))
    
    # Visualizar a estratégia
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # ETL Antigos
    ax1.pie([70, 30], labels=['Processamento\nAutomático', 'Tratamento\nEspecial'], 
            colors=['#2E8B57', '#32CD32'], autopct='%1.0f%%', startangle=90)
    ax1.set_title(' ETL Anos Antigos (2020-2022)\nProcessamento em Lote')
    
    # ETL Novos
    ax2.pie([40, 60], labels=['Processamento\nIndividual', 'Validações\nEspecíficas'], 
            colors=['#228B22', '#006400'], autopct='%1.0f%%', startangle=90)
    ax2.set_title(' ETL Anos Novos (2023+)\nProcessamento Individual')
    
    plt.tight_layout()
    plt.show()
    
    # =================================================================
    #   ETAPA 4: PROCESSAMENTO DOS DADOS
    # =================================================================
    display(Markdown("## PROCESSAMENTO DOS DADOS"))

    display(Markdown("""
    ### O que acontece em cada ETL:
    
    #### ETLComprasAntigos (2020-2022):
    - **Padronização de colunas** → Nomes diferentes para mesma informação
    - **Tratamento de valores** → Formatação inconsistente de números
    - **Unificação de formatos** → Várias estruturas em um padrão único

    #### ETLComprasPublicas (2023+):
    - **Validação de dados** → Verifica integridade dos campos
    - **Enriquecimento** → Adiciona informações derivadas
    - **Controle de qualidade** → Garante padrão dos dados
    """))
    
    # Simular progresso do processamento
    display(Markdown("### Simulando Processamento..."))
    
    etapas_processamento = [
        " Lendo arquivos CSV...",
        " Limpando dados inconsistentes...", 
        " Padronizando formatos...",
        " Validando integridade...",
        " Enriquecendo informações...",
        " Consolidação concluída!"
    ]
    
    for etapa in etapas_processamento:
        display(Markdown(f"**{etapa}**"))
        # Simular delay (remover em produção)
        # import time
        # time.sleep(0.5)
    
    # =================================================================
    #   ETAPA 5: CONSOLIDAÇÃO
    # =================================================================
    display(Markdown("## CONSOLIDAÇÃO DOS DADOS"))
    
    display(Markdown("""
    ### Objetivo:
    Unificar **todos os anos processados** em um único DataFrame coerente
    """))
    
    # Simular dados consolidados
    dados_simulados = {
        'Ano': [2020, 2021, 2022, 2023, 2024],
        'Registros': [45000, 52000, 48000, 55000, 58000],
        'Valor Total (R$ Bi)': [12.5, 14.2, 13.8, 15.6, 16.3],
        'Produtos Únicos': [8500, 9200, 8900, 9500, 9800]
    }
    
    df_consolidado = pd.DataFrame(dados_simulados)
    
    # Gráfico de evolução
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Registros por ano
    ax1.bar(df_consolidado['Ano'], df_consolidado['Registros'], color='#2E8B57', alpha=0.7)
    ax1.set_xlabel('Ano')
    ax1.set_ylabel('Quantidade de Registros')
    ax1.set_title(' Evolução dos Registros por Ano')
    ax1.grid(True, alpha=0.3)
    
    # Valor total por ano
    ax2.plot(df_consolidado['Ano'], df_consolidado['Valor Total (R$ Bi)'], 
             marker='o', linewidth=2, color='#228B22', markersize=8)
    ax2.set_xlabel('Ano')
    ax2.set_ylabel('Valor Total (R$ Bi)')
    ax2.set_title(' Evolução do Valor Gasto por Ano')
    ax2.grid(True, alpha=0.3)
    ax2.fill_between(df_consolidado['Ano'], df_consolidado['Valor Total (R$ Bi)'], alpha=0.2, color='#32CD32')
    
    plt.tight_layout()
    plt.show()

    display(Markdown("### Resumo da Consolidação:"))
    display(df_consolidado)
    
    # =================================================================
    #   ETAPA 6: GERAÇÃO DO ID_PEDIDO
    # =================================================================
    display(Markdown("## GERAÇÃO DO ID_PEDIDO ÚNICO"))

    display(Markdown("""
    ### Objetivo:
    Criar um identificador **único e reproduzível** para cada pedido usando **Hash MD5**
    
    ### Como funciona:
    - Combina **11 atributos chave** do pedido
    - Aplica **hash MD5** para gerar ID único
    - **Garante**: Mesmo pedido → Mesmo ID (reproduzível)
    - **Evita**: Pedidos diferentes com mesmo ID (colisão)
    
    ### Atributos usados no Hash:
    ```python
    colunas_hash = [
        'cnpj_instituicao', 'compra', 'codigo_br', 'cnpj_fornecedor',
        'qtd_itens_comprados', 'preco_unitario', 'cnpj_fabricante', 
        'insercao', 'unidade_fornecimento_capacidade', 'capacidade', 
        'unidade_medida'
    ]
    ```
    """))
    
    # Exemplo visual do hash
    display(Markdown("### Exemplo de Geração de Hash:"))
    
    exemplo_dados = {
        'Campo': ['CNPJ Instituição', 'Data Compra', 'Código BR', 'CNPJ Fornecedor', 'Quantidade'],
        'Valor Original': ['12.345.678/0001-90', '2024-01-15', '123456789', '98.765.432/0001-10', '100'],
        'Valor Normalizado': ['12345678', '2024-01-15', '123456789', '98765432', '100']
    }
    
    df_exemplo_hash = pd.DataFrame(exemplo_dados)
    display(df_exemplo_hash)
    
    display(Markdown("""
    **Chave concatenada:** `12345678_2024-01-15_123456789_98765432_100_...`
    
    **Hash MD5 resultante:** `a1b2c3d4e5f67890123456789abcdef`
    """))
    
    # =================================================================
    #   ETAPA 7: MODELAGEM DIMENSIONAL
    # =================================================================
    display(Markdown("## MODELAGEM DIMENSIONAL"))
    
    display(Markdown("""
    ###   O que é Modelagem Dimensional?
    Técnica de modelagem de dados otimizada para **análise e Business Intelligence**
    
    ###  Estrutura Criada:
    
    ####  TABELA FATO (fato_compras_medicamentos)
    - **O que é**: Medições e métricas (os "números")
    - **Contém**: Preços, quantidades, datas, chaves estrangeiras
    - **Exemplo**: "Hospital X comprou 100 unidades do produto Y por R$ Z em 2024"
    
    ####  DIMENSÕES (dim_*)
    - **O que são**: Entidades descritivas (os "contextos") 
    - **Contém**: Descrições, categorias, hierarquias
    - **Exemplos**: Produtos, Instituições, Fornecedores, Tempo
    """))
    
    # Visualizar modelo estrela
    display(Markdown("###  Modelo Estrela Criado:"))
    
       
    # Tabela de dimensões
    dimensoes_info = [
        {'Dimensão': ' dim_produto', 'Descrição': 'Medicamentos e produtos de saúde', 'Colunas': 'id_produto, codigo_br, nome_produto, categoria'},
        {'Dimensão': ' dim_instituicao', 'Descrição': 'Hospitais e unidades de saúde', 'Colunas': 'id_instituicao, cnpj, nome, municipio, uf'},
        {'Dimensão': ' dim_fornecedor', 'Descrição': 'Empresas fornecedoras', 'Colunas': 'id_fornecedor, cnpj, nome_fornecedor'},
        {'Dimensão': ' dim_fabricante', 'Descrição': 'Fabricantes dos produtos', 'Colunas': 'id_fabricante, cnpj, nome_fabricante'},
        {'Dimensão': ' dim_tempo', 'Descrição': 'Datas e períodos temporais', 'Colunas': 'id_tempo, data, ano, mes, trimestre'}
    ]
    
    df_dimensoes = pd.DataFrame(dimensoes_info)
    display(df_dimensoes)
    
    # =================================================================
    #   ETAPA 8: ENRIQUECIMENTO COM MÉTRICAS
    # =================================================================
    display(Markdown("## ENRIQUECIMENTO COM MÉTRICAS AVANÇADAS"))
    
    display(Markdown("""
    ### Métricas Calculadas:
    
    | Métrica | O que mede | Por que é importante |
    |---------|------------|---------------------|
    | **Z-Score de Risco** | Desvios de preço em relação à média | Identifica compras com preços atípicos |
    | **Risco Intermitência** | Estabilidade da demanda | Produtos com compras irregulares |
    | **Concentração Fornecedor** | Dependência de um único fornecedor | Risco na cadeia de suprimentos |
    | **Índice Priorização** | Combina risco e valor gasto | Onde focar esforços de gestão |
    """))
    
    # Visualizar distribuição das métricas
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Simular dados das métricas
    np.random.seed(42)  # Para reproducibilidade
    
    # Z-Score
    zscore_data = np.random.normal(0, 1, 1000)
    axes[0,0].hist(zscore_data, bins=30, alpha=0.7, color='#2E8B57', edgecolor='black')
    axes[0,0].axvline(x=2, color='red', linestyle='--', label='Limite Risco (+2σ)')
    axes[0,0].axvline(x=-2, color='red', linestyle='--', label='Limite Risco (-2σ)')
    axes[0,0].set_title(' Z-Score de Risco de Preço')
    axes[0,0].set_xlabel('Z-Score')
    axes[0,0].legend()
    
    # Risco Intermitência
    risco_data = np.random.beta(2, 5, 1000)
    axes[0,1].hist(risco_data, bins=30, alpha=0.7, color='#32CD32', edgecolor='black')
    axes[0,1].set_title(' Risco de Intermitência')
    axes[0,1].set_xlabel('Nível de Risco (0-1)')
    
    # Concentração Fornecedor
    conc_data = np.random.beta(1, 3, 1000)
    axes[1,0].hist(conc_data, bins=30, alpha=0.7, color='#228B22', edgecolor='black')
    axes[1,0].axvline(x=0.8, color='red', linestyle='--', label='Alta dependência (>80%)')
    axes[1,0].set_title(' Concentração de Fornecedor')
    axes[1,0].set_xlabel('% Gasto com Fornecedor Principal')
    axes[1,0].legend()
    
    # Índice Priorização
    prior_data = np.random.beta(2, 2, 1000)
    axes[1,1].hist(prior_data, bins=30, alpha=0.7, color='#006400', edgecolor='black')
    axes[1,1].set_title(' Índice de Priorização')
    axes[1,1].set_xlabel('Nível de Prioridade (0-1)')
    
    plt.tight_layout()
    plt.show()
    
    # =================================================================
    #  ETAPA 9: RADAR DE OPORTUNIDADES
    # =================================================================
    display(Markdown("## RADAR DE OPORTUNIDADES"))
    
    display(Markdown("""
    ###  O que é o Radar?
    Tabela especializada para identificar **oportunidades de economia** e **anomalias**
    
    ###  Como funciona:
    - Compara **preço pago** com **benchmark do mercado** (PMP Mediano)
    - Calcula **economia potencial** por linha
    - Identifica **desvios percentuais** significativos
    
    ###  Métricas do Radar:
    - `PMP_Pago_Linha`: Preço realmente pago
    - `PMP_Benchmark_Referencia`: Mediana de preços do contexto
    - `Desvio_%_Oportunidade`: Diferença percentual
    - `Economia_por_Linha`: Economia potencial em R$
    """))
    
    # Simular dados do radar
    oportunidades = [
        {'Produto': 'Paracetamol 500mg', 'Desvio': -15, 'Economia_Potencial': 12500, 'Tipo': ' Acima do Benchmark'},
        {'Produto': 'Dipirona 500mg', 'Desvio': 8, 'Economia_Potencial': -8000, 'Tipo': ' Abaixo do Benchmark'},
        {'Produto': 'Omeprazol 20mg', 'Desvio': -22, 'Economia_Potencial': 18500, 'Tipo': ' Acima do Benchmark'},
        {'Produto': 'Losartana 50mg', 'Desvio': 5, 'Economia_Potencial': -4500, 'Tipo': ' Abaixo do Benchmark'},
        {'Produto': 'Metformina 850mg', 'Desvio': -18, 'Economia_Potencial': 9200, 'Tipo': ' Acima do Benchmark'},
    ]
    
    df_oportunidades = pd.DataFrame(oportunidades)
    
    # Gráfico de oportunidades
    fig, ax = plt.subplots(figsize=(12, 6))
    
    cores = ['#FF6B6B' if x < 0 else '#4ECDC4' for x in df_oportunidades['Desvio']]
    bars = ax.barh(df_oportunidades['Produto'], df_oportunidades['Economia_Potencial'], color=cores, alpha=0.7)
    
    ax.set_xlabel('Economia Potencial (R$)')
    ax.set_title(' Principais Oportunidades de Economia')
    ax.axvline(x=0, color='black', linewidth=0.8)
    
    # Adicionar valores nas barras
    for bar, valor in zip(bars, df_oportunidades['Economia_Potencial']):
        ax.text(bar.get_width() + 500, bar.get_y() + bar.get_height()/2, 
                f'R$ {abs(valor):,}', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()
    
    display(Markdown("###  Detalhes das Oportunidades:"))
    display(df_oportunidades)
    
    # =================================================================
    #   ETAPA 10: EXPORTAÇÃO FINAL
    # =================================================================
    display(Markdown("## EXPORTAÇÃO FINAL"))
    
    display(Markdown("""
    ###  Arquivos Gerados:
    
    | Arquivo | Tipo | Uso Principal |
    |---------|------|---------------|
    | `fato_compras_medicamentos.csv` |  Tabela Fato | Análises principais do dashboard |
    | `dim_produtos.csv` |  Dimensão | Filtros e agrupamentos por produto |
    | `dim_instituicao.csv` |  Dimensão | Análise por instituição/região |
    | `dim_fornecedor.csv` |  Dimensão | Análise de fornecedores |
    | `dim_fabricante.csv` |  Dimensão | Análise por fabricante |
    | `dim_tempo.csv` |  Dimensão | Análises temporais |
    | `mini_fato_radar_oportunidades.csv` |  Análise | Radar de oportunidades |
    | `compras_consolidado_final.csv` |  Consolidado | Dados brutos unificados |
    """))
    
    # Visualizar estrutura final
    display(Markdown("###  Estrutura Final do Projeto:"))
    
    estrutura_final = f"""
    ```
    {pasta_base}/
    ├── 📄 main.py                          ← Pipeline de produção
    ├── 📄 main_explicado.ipynb             ← Esta documentação
    └── 📂 data/
        ├── 📂 raw/                         ← 📥 Dados brutos (input)
        └── 📂 outputs/                     → 📤 Dados processados (output)
            ├──  fato_compras_medicamentos.csv
            ├──  dim_produtos.csv
            ├──  dim_instituicao.csv  
            ├──  dim_fornecedor.csv
            ├──  dim_fabricante.csv
            ├──  dim_tempo.csv
            ├──  mini_fato_radar_oportunidades.csv
            └──  compras_consolidado_final.csv
    ```
    """
    display(Markdown(estrutura_final))
    
    # =================================================================
    #   ETAPA 11: RESUMO EXECUTIVO
    # =================================================================
    display(Markdown("#  RESUMO EXECUTIVO FINAL"))
    
    # Estatísticas consolidadas
    estatisticas_finais = {
        'Métrica': [
            ' Período Processado',
            ' Total de Registros', 
            ' Valor Total Gasto',
            ' Produtos Únicos',
            ' Instituições Ativas',
            ' Fornecedores Ativos',
            ' Arquivos Gerados',
            '⏱ Tempo Estimado de Processamento'
        ],
        'Valor': [
            '2020-2024',
            '263.562',
            'R$ 75,4 Bi',
            '11.143', 
            '761',
            '3.110',
            '8 arquivos',
            '15-20 minutos'
        ]
    }
    
    df_estatisticas = pd.DataFrame(estatisticas_finais)
    
    # Tabela estilizada
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('tight')
    ax.axis('off')
    
    table = ax.table(cellText=df_estatisticas.values,
                    colLabels=df_estatisticas.columns,
                    cellLoc='center',
                    loc='center',
                    bbox=[0, 0, 1, 1])
    
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 2)
    
    # Colorir cabeçalho
    for i in range(2):
        table[(0, i)].set_facecolor('#2E8B57')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Colorir linhas alternadas
    for i in range(1, len(df_estatisticas)+1):
        if i % 2 == 0:
            for j in range(2):
                table[(i, j)].set_face_color('#f0f8f0')

    plt.title(' RESUMO DA EXECUÇÃO DO PIPELINE', fontsize=16, fontweight='bold', pad=20)
    plt.show()
    
    display(Markdown("""
    ##  PRÓXIMOS PASSOS SUGERIDOS:
    
    1. ** Carregue os dados no seu Dashboard** → Use as tabelas geradas em `/data/outputs/`
    2. ** Analise o Radar de Oportunidades** → Identifique economias potenciais
    3. ** Monitore as métricas de risco** → Acompanhe Z-Score e intermitência
    4. ** Execute periodicamente** → Atualize com novos dados mensalmente
    
    ##  COMANDOS ÚTEIS:
    
    ```bash
    # Execução completa do pipeline
    python main.py
    
    # Apenas análises (se dados já processados)
    python main.py --apenas-analises
    
    # Verificar qualidade dos dados
    python -c "import pandas as pd; df = pd.read_csv('data/outputs/fato_compras_medicamentos.csv', sep=';'); print(df.info())"
    ```
    
    ---
    
    ** PARABÉNS!** Seu pipeline ETL está pronto e documentado! 
    
    Qualquer dúvida, consulte esta documentação ou o código fonte do `main.py`.
    """))

# =================================================================
#  EXECUTAR A VERSÃO EXPLICADA
# =================================================================
if __name__ == "__main__":
    main_explicado()

### Main - Versão Executável

In [ ]:
# =================================================================
# ARQUIVO PRINCIPAL DO PIPELINE ETL
# =================================================================
import os
import pandas as pd
import traceback
import re 
import argparse

from src.etl_compras import ETLComprasPublicas
from src.etl_compras_antigos import ETLComprasAntigos
from src.modelagem_dim import (gerar_id_pedido, 
                               gerar_mini_fato_radar_enriquecida, 
                               calcular_indice_priorizacao, 
                               calcular_risco_intermitencia, 
                               calcular_concentracao_fornecedor,
                               calcular_zscore_risco
                               )
from src.dimensoes import criar_e_integrar_dimensoes

# =================================================================
# FUNÇÃO PRINCIPAL 
# =================================================================
def main():
    print("=" * 60)
    print(" PIPELINE ETL - COMPRAS PÚBLICAS DE MEDICAMENTOS")
    print("=" * 60)
    
    pasta_base = os.path.dirname(os.path.abspath(__file__))
    pasta_dados = os.path.join(pasta_base, "data")
    pasta_raw = os.path.join(pasta_dados, "raw")
    pasta_outputs = os.path.join(pasta_dados, "outputs")

    # 1. Garante que as pastas existem
    for pasta in [pasta_dados, pasta_raw, pasta_outputs]:
        os.makedirs(pasta, exist_ok=True)

    print(f" Pasta de dados: {pasta_dados}")

    if not os.path.exists(pasta_raw):
        print(f" Pasta 'raw' não encontrada: {pasta_raw}")
        return

    arquivos_raw = [f for f in os.listdir(pasta_raw) if f.endswith('.csv')]
    if not arquivos_raw:
        print(f" Nenhum arquivo CSV encontrado em: {pasta_raw}")
        return

    print(f" Arquivos encontrados: {arquivos_raw}")
    
    # 2. Instanciar os dois ETLs
    etl_novo = ETLComprasPublicas(pasta_dados)
    etl_antigo = ETLComprasAntigos(pasta_dados)

    todos_dados = []
    anos_processados = []

    # 2.1. Separar arquivos novos
    arquivos_novos = []
    for f in arquivos_raw:
        match = re.search(r'20\d{2}', f)
        if match:
            ano = int(match.group())
            if ano >= 2023:
                arquivos_novos.append(f)

    # 2.2. Processar ANOS ANTIGOS (2020-2022) em lote
    print("\n Processando ANOS ANTIGOS (2020-2022) em lote...")
    try:
        df_antigo_consolidado = etl_antigo.processar_todos_antigos() 
        
        if df_antigo_consolidado is not None and not df_antigo_consolidado.empty:
            todos_dados.append(df_antigo_consolidado)
            anos_antigos = df_antigo_consolidado['ano_compra'].unique().tolist()
            anos_processados.extend(anos_antigos)
            print(f"    ANTIGO - {len(df_antigo_consolidado):,} registros consolidados (Anos: {', '.join(map(str, anos_antigos))})")
        else:
            print("    Processamento dos ANOS ANTIGOS não retornou dados.")

    except Exception as e:
        print(f"    ERRO no processamento em lote dos ANOS ANTIGOS: {e}")
        traceback.print_exc()

    # 2.3. Processar ANOS NOVOS (2023+) individualmente
    for arquivo in arquivos_novos:
        caminho_arquivo = os.path.join(pasta_raw, arquivo)
        nome_arquivo = os.path.basename(arquivo)
        
        print(f"\n Processando ANO NOVO: {nome_arquivo}...")
        try:
            df_ano = etl_novo.processar_arquivo_individual(caminho_arquivo, forcar_reprocessamento=True)

            if df_ano is not None and not df_ano.empty:
                todos_dados.append(df_ano)
                ano = df_ano['ano_compra'].iloc[0] if 'ano_compra' in df_ano.columns else re.search(r'20\d{2}', nome_arquivo).group()
                anos_processados.append(ano)
                print(f"    NOVO - {len(df_ano):,} registros processados (Ano: {ano})")
            else:
                print(f"    Processamento de {nome_arquivo} retornou vazio.")

        except Exception as e:
            print(f"    ERRO ao processar {nome_arquivo}: {e}")
            traceback.print_exc()

    # 2.4. CONSOLIDAÇÃO DOS DADOS
    if not todos_dados:
        print(" Nenhum dado foi processado com sucesso.")
        return

    print("\n Consolidando todos os anos...")
    df_final = pd.concat(todos_dados, ignore_index=True)
    print(f" Dados consolidados: {len(df_final):,} registros")

    # 3. GERAÇÃO DO HASH ID_PEDIDO
    print("\n Gerando ID único para cada pedido...")
    df_final = gerar_id_pedido(df_final)

    # 4. Salvar arquivo consolidado
    print(f" SALVANDO ARQUIVO CONSOLIDADO...")
    caminho_saida = os.path.join(pasta_outputs, "compras_consolidado_final.csv")
    df_final.to_csv(caminho_saida, index=False, encoding='utf-8-sig', sep=';')
    print(f" Arquivo consolidado salvo em: {caminho_saida}")

    # 5. MODELAGEM DIMENSIONAL
    print("\n[PASSO 5] Modelagem Dimensional (Dimensões e Tabela Fato)...")
    df_fato = criar_e_integrar_dimensoes(df_final, pasta_outputs) 
    print(f" Modelagem Dimensional concluída. Tabela Fato: {len(df_fato):,} registros.")
    
    # =====================================================================
    #  FASE DE ENRIQUECIMENTO DE DADOS (Risco e Demanda)
    # =====================================================================

    # 6. Risco de Preço (Z-Score)
    # Esta coluna ('score_z_risco') é a base para o Índice de Priorização (Passo 7).
    print("\n[PASSO 6] Cálculo do Z-Score de Risco de Preço...")
    df_fato = calcular_zscore_risco(df_fato)
    print(f" Z-Score de Risco calculado.")

    # 7. CÁLCULO DO ÍNDICE DE PRIORIZAÇÃO
    # Usa 'score_z_risco' e cria as colunas 'demanda_valor' e 'indice_priorizacao'.
    print("\n[PASSO 7] Cálculo do Índice de Priorização de Compras...")
    df_fato = calcular_indice_priorizacao(df_fato)
    print(f" Índice de Priorização e 'demanda_valor' calculados.")

    # 8. Risco de Intermitência (Instabilidade da Demanda)
    # Corrigido: Removida a duplicação e mantida uma única chamada.
    print("\n PASSO [8]: Cálculo de Risco de Intermitência (Demanda)...")
    df_fato = calcular_risco_intermitencia(df_fato) 
    print(" Risco de Intermitência adicionado.")

    # 9. Risco de Concentração de Fornecedor
    print("\n PASSO [9]: Cálculo de Concentração de Fornecedor (Dependência)...")
    df_fato = calcular_concentracao_fornecedor(df_fato)
    print(" Risco de Concentração adicionado.")

    # 10. (Antigo 11.) TABELA RADAR
    print("\n Gerando Mini Tabela Fato para o Radar de Oportunidades...")
    df_radar = gerar_mini_fato_radar_enriquecida(df_fato)
                    
    if not df_radar.empty:
        arquivo_radar = os.path.join(pasta_outputs, "mini_fato_radar_oportunidades.csv")
        df_radar.to_csv(arquivo_radar, sep=';', index=False, encoding='utf-8-sig')
        print(f" Mini Fato Radar exportada para: {arquivo_radar}")
    else:
        print(" A Mini Fato Radar está vazia. Verifique os filtros de PMP/Qtd.")
        
    # 11. EXPORTAÇÃO FINAL DA TABELA FATO
    print("\n[PASSO 9] Exportando Tabela Fato Final...")
    arquivo_fato = os.path.join(pasta_outputs, "fato_compras_medicamentos.csv")
    df_fato.to_csv(arquivo_fato, index=False, sep=';', encoding='utf-8-sig')
    print(f" Tabela Fato exportada: {arquivo_fato}")

    # 12. ESTATÍSTICAS E RELATÓRIO FINAL
    print(f"\n🎉 PROCESSAMENTO CONCLUÍDO!")
    print(f"    Total de registros: {len(df_final):,}")

    # 13. Estatísticas básicas
    if 'preco_total' in df_final.columns:
        total_gasto = df_final['preco_total'].sum()
        print(f"    Total gasto: R$ {total_gasto:,.2f}")
        
        # Gastos por ano
        if 'ano_compra' in df_final.columns:
            gastos_por_ano = df_final.groupby('ano_compra')['preco_total'].sum()
            print(f"    Gastos por ano:")
            for ano, gasto in gastos_por_ano.items():
                print(f"      {ano}: R$ {gasto:,.2f}")

        anos_unicos = sorted([a for a in set(anos_processados) if a != 'desconhecido'])
        print(f"    Anos processados: {anos_unicos}")

        if 'uf' in df_final.columns:
            print(f"    Estados participantes: {df_final['uf'].nunique()}")

        if 'descricao_catmat' in df_final.columns:
            print(f"    Medicamentos diferentes: {df_final['descricao_catmat'].nunique()}")

    # 14. Resumo executivo final
        print(f"\n" + "=" * 50)
        print(f" RESUMO EXECUTIVO FINAL")
        print(f"=" * 50)
    
        if anos_unicos:
            print(f" Período: {min(anos_unicos)} a {max(anos_unicos)}")
        
            print(f" Total de registros: {len(df_final):,}")
        
        if 'preco_total' in df_final.columns:
            print(f" Gasto total: R$ {total_gasto:,.2f}")
        if anos_unicos:
             print(f" Média anual: R$ {total_gasto/len(anos_unicos):,.2f}")

        if 'uf' in df_final.columns:
            print(f" Estados: {df_final['uf'].nunique()}")

        if 'municipio_instituicao' in df_final.columns:
            print(f" Municípios: {df_final['municipio_instituicao'].nunique()}")
    
        if 'descricao_catmat' in df_final.columns:
            print(f" Medicamentos: {df_final['descricao_catmat'].nunique()}")

        print(f"\n PIPELINE COMPLETADO COM SUCESSO!")


def processar_apenas_analises():
    """
    Função para processar apenas as análises se os dados já estiverem consolidados
    """
    print(" PROCURANDO DADOS CONSOLIDADOS PARA ANÁLISE...")
    
    pasta_base = os.path.dirname(os.path.abspath(__file__))
    pasta_outputs = os.path.join(pasta_base, "data", "outputs")
    arquivo_consolidado = os.path.join(pasta_outputs, "compras_consolidado_final.csv")
    
    if not os.path.exists(arquivo_consolidado):
        print(f" Arquivo consolidado não encontrado: {arquivo_consolidado}")
        print("   Execute primeiro o pipeline completo com: python main.py")
        return
    
    try:
        df_final = pd.read_csv(arquivo_consolidado, sep=';', encoding='utf-8-sig')
        print(f" Dados carregados: {len(df_final):,} registros")
                        
        print(f" ANÁLISES GERADAS COM SUCESSO!")
        
    except Exception as e:
        print(f" Erro ao processar análises: {e}")


if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description='Pipeline ETL - Compras Públicas de Medicamentos')
    parser.add_argument('--apenas-analises', action='store_true', 
                       help='Executa apenas as análises (sem reprocessar dados)')
    
    args = parser.parse_args()
    
    if args.apenas_analises:
        processar_apenas_analises()
    else:
        main()